<h1>Trabalho 1 de SAD</h1>

<h3>1 - Posse de bola influencia no resultado final da partida?</h3>
<h4>Banco:</h4> 'Match' <h4>Colunas:</h4> 'home_team_goal', 'away_team_goal', 'possession'

<h3>2 - Time que menos modifica seus jogadores tem melhor colocacao na tabela?</h3>
<h4>Banco:</h4> 'Match' <h4>Colunas:</h4> 'home_player[1..11]', 'match_api_id', 'session', 'home_team_api_id'\
       'away_team_api_id', 'away_player[1..11]', 'home_team_goal', 'away_team_goal'

<h3>3 - Time que mais chuta ao gol influencia no resultado final da partida?</h3>
<h4>Banco:</h4> 'Match' <h4>Colunas:</h4> 'home_team_goal', 'away_team_goal', 'shoton', 'shotoff'

In [16]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

path_banco = os.getenv("HOME") + '/Bancos_Dados/UFJF/Periodo_2019_1/SAD/'

In [17]:
#Bases

#base_Country = pd.read_csv(path_banco + 'Country.csv')
#base_League = pd.read_csv(path_banco + 'League.csv')
base_Match = pd.read_csv(path_banco + 'Match.csv',low_memory = False)
#base_Player = pd.read_csv(path_banco + 'Player.csv')
#base_Player_Attributes = pd.read_csv(path_banco + 'Player_Attributes.csv')
#base_Team= pd.read_csv(path_banco + 'Team.csv')
#base_Team_Attributes= pd.read_csv(path_banco + 'Team_Attributes.csv')
#base_sqlite_sequence= pd.read_csv(path_banco + 'sqlite_sequence.csv')


In [18]:
#Arrumando a base_Match
coluna = base_Match.loc[:,'possession']
coluna1 = base_Match.loc[pd.notnull(base_Match['possession'])]
coluna1 = coluna1.loc[:,['possession']]
coluna2 = coluna1.values

matriz = [] #Matriz que armazena os vetores resultados de cada parser
for linha in coluna:
    soup = BeautifulSoup(str(linha), 'html.parser')
    awaypos = soup.find_all('awaypos')
    homepos = soup.find_all('homepos')

    awaypos_num = []
    for pos in awaypos:
        awaypos_num.append(pos.get_text())
    homepos_num = []
    for pos in homepos:
        homepos_num.append(pos.get_text())
    #print(awaypos_num[1000])
    #print(homepos_num[1000])
    matriz.append([homepos_num,awaypos_num])
#A ideia aqui eh criar duas colunas para possession, onde teremos possession_home e possession_away
print(matriz[10000])

[['50', '50', '49', '49', '49', '49', '49', '49'], ['50', '50', '51', '51', '51', '51', '51', '51']]


In [19]:
base1 = base_Match.loc[:,['home_team_goal','away_team_goal','possession']]

base2 = base_Match.loc[:,['home_player_1','home_player_2','home_player_3','home_player_4','home_player_5',
                         'home_player_6','home_player_7','home_player_8','home_player_9','home_player_10','home_player_11',
                         'away_player_1','away_player_2','away_player_3','away_player_4','away_player_5','away_player_6',
                         'away_player_7','away_player_8','away_player_9','away_player_10','away_player_11', 'match_api_id',
                         'session','home_team_api_id','away_team_api_id','home_team_goal','away_team_goal']]

base3 = base_Match.loc[:,['home_team_goal', 'away_team_goal', 'shoton', 'shotoff']]

/home/bruno/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [20]:
base1.mean() #Media

home_team_goal    1.544594
away_team_goal    1.160938
dtype: float64

In [21]:
base2.mean() #Media

home_player_1       7.663836e+04
home_player_2       1.068541e+05
home_player_3       9.160129e+04
home_player_4       9.454020e+04
home_player_5       1.095278e+05
home_player_6       1.023088e+05
home_player_7       9.728764e+04
home_player_8       1.072910e+05
home_player_9       1.111315e+05
home_player_10      1.056125e+05
home_player_11      1.034139e+05
away_player_1       7.662820e+04
away_player_2       1.076146e+05
away_player_3       9.112678e+04
away_player_4       9.508391e+04
away_player_5       1.098009e+05
away_player_6       1.023083e+05
away_player_7       9.789806e+04
away_player_8       1.092651e+05
away_player_9       1.110870e+05
away_player_10      1.071491e+05
away_player_11      1.049329e+05
match_api_id        1.195429e+06
session                      NaN
home_team_api_id    9.984372e+03
away_team_api_id    9.984475e+03
home_team_goal      1.544594e+00
away_team_goal      1.160938e+00
dtype: float64

In [22]:
base3.mean() # Media

home_team_goal    1.544594
away_team_goal    1.160938
dtype: float64

In [23]:
base_Match.describe() #Estatisticas

,id,country_id,league_id,stage,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
count,25979.000000,25979.000000,25979.000000,25979.000000,2.597900e+04,25979.000000,25979.000000,25979.000000,25979.000000,24158.000000,...,17097.000000,22568.000000,22568.000000,22568.000000,14162.000000,14162.000000,14162.000000,14161.000000,14161.000000,14161.000000
mean,12990.000000,11738.630317,11738.630317,18.242773,1.195429e+06,9984.371993,9984.475115,1.544594,1.160938,0.999586,...,4.622343,2.668107,3.899048,4.840281,2.498764,3.648189,4.353097,2.497894,3.660742,4.405663
std,7499.635658,7553.936759,7553.936759,10.407354,4.946279e+05,14087.453758,14087.445135,1.297158,1.142110,0.022284,...,3.632164,1.928753,1.248221,4.318338,1.489299,0.867440,3.010189,1.507793,0.868272,3.189814
min,1.000000,1.000000,1.000000,1.000000,4.831290e+05,1601.000000,1601.000000,0.000000,0.000000,0.000000,...,1.100000,1.030000,1.620000,1.080000,1.050000,1.450000,1.120000,1.040000,1.330000,1.120000
25%,6495.500000,4769.000000,4769.000000,9.000000,7.684365e+05,8475.000000,8475.000000,1.000000,0.000000,1.000000,...,2.500000,1.700000,3.300000,2.550000,1.670000,3.200000,2.500000,1.670000,3.250000,2.500000
50%,12990.000000,10257.000000,10257.000000,18.000000,1.147511e+06,8697.000000,8697.000000,1.000000,1.000000,1.000000,...,3.500000,2.150000,3.500000,3.500000,2.100000,3.300000,3.400000,2.100000,3.400000,3.400000
75%,19484.500000,17642.000000,17642.000000,27.000000,1.709852e+06,9925.000000,9925.000000,2.000000,2.000000,1.000000,...,5.250000,2.800000,4.000000,5.400000,2.650000,3.750000,5.000000,2.620000,3.750000,5.000000
max,25979.000000,24558.000000,24558.000000,38.000000,2.216672e+06,274581.000000,274581.000000,10.000000,9.000000,2.000000,...,41.000000,36.000000,26.000000,67.000000,21.000000,11.000000,34.000000,17.000000,13.000000,34.000000


In [24]:
base_Match.columns #Nomes Colunas

Index(['id', 'country_id', 'league_id', 'season', 'stage', 'date',
       'match_api_id', 'home_team_api_id', 'away_team_api_id',
       'home_team_goal',
       ...
       'SJA', 'VCH', 'VCD', 'VCA', 'GBH', 'GBD', 'GBA', 'BSH', 'BSD', 'BSA'],
      dtype='object', length=115)

In [25]:
base_Match.shape #Linhas x Colunas

(25979, 115)

In [26]:
base_Match.count() #Count

id                  25979
country_id          25979
league_id           25979
season              25979
stage               25979
date                25979
match_api_id        25979
home_team_api_id    25979
away_team_api_id    25979
home_team_goal      25979
away_team_goal      25979
home_player_X1      24158
home_player_X2      24158
home_player_X3      24147
home_player_X4      24147
home_player_X5      24147
home_player_X6      24147
home_player_X7      24147
home_player_X8      24147
home_player_X9      24147
home_player_X10     24147
home_player_X11     24147
away_player_X1      24147
away_player_X2      24147
away_player_X3      24147
away_player_X4      24147
away_player_X5      24147
away_player_X6      24147
away_player_X7      24147
away_player_X8      24147
                    ...  
B365H               22592
B365D               22592
B365A               22592
BWH                 22575
BWD                 22575
BWA                 22575
IWH                 22520
IWD         

In [27]:
base_Match.min() #Valor Minimo

id                                    1
country_id                            1
league_id                             1
season                        2008/2009
stage                                 1
date                2008-07-18 00:00:00
match_api_id                     483129
home_team_api_id                   1601
away_team_api_id                   1601
home_team_goal                        0
away_team_goal                        0
home_player_X1                        0
home_player_X2                        0
home_player_X3                        1
home_player_X4                        2
home_player_X5                        1
home_player_X6                        1
home_player_X7                        1
home_player_X8                        1
home_player_X9                        1
home_player_X10                       1
home_player_X11                       1
away_player_X1                        1
away_player_X2                        1
away_player_X3                        2


In [28]:
base_Match.max() #Valor Maximo

id                                25979
country_id                        24558
league_id                         24558
season                        2015/2016
stage                                38
date                2016-05-25 00:00:00
match_api_id                    2216672
home_team_api_id                 274581
away_team_api_id                 274581
home_team_goal                       10
away_team_goal                        9
home_player_X1                        2
home_player_X2                        8
home_player_X3                        8
home_player_X4                        8
home_player_X5                        9
home_player_X6                        9
home_player_X7                        9
home_player_X8                        9
home_player_X9                        9
home_player_X10                       9
home_player_X11                       7
away_player_X1                        6
away_player_X2                        8
away_player_X3                        9


In [29]:
base_Match.median() #Mediana

id                    12990.00
country_id            10257.00
league_id             10257.00
stage                    18.00
match_api_id        1147511.00
home_team_api_id       8697.00
away_team_api_id       8697.00
home_team_goal            1.00
away_team_goal            1.00
home_player_X1            1.00
home_player_X2            2.00
home_player_X3            4.00
home_player_X4            6.00
home_player_X5            8.00
home_player_X6            3.00
home_player_X7            5.00
home_player_X8            6.00
home_player_X9            5.00
home_player_X10           5.00
home_player_X11           6.00
away_player_X1            1.00
away_player_X2            2.00
away_player_X3            4.00
away_player_X4            6.00
away_player_X5            8.00
away_player_X6            3.00
away_player_X7            5.00
away_player_X8            6.00
away_player_X9            5.00
away_player_X10           5.00
                       ...    
B365H                     2.10
B365D   